In [1]:
import pandas as pd
import numpy as np
import os.path
import recsys as rs
import notipy
import json
import math
import sys
import TopSimilarRecommender as TSR
import ItemBasedRecommender as IBR
import UserBasedRecommender as UBR
import random
from scipy import sparse as sps
import importlib

# Test split

In [2]:
data = pd.read_csv('Data/train_final.csv','\t')
tracks = pd.read_csv('Data/tracks_final.csv','\t')
pl_info = pd.read_csv('Data/playlists_final.csv','\t')
tgt_playlists = pd.read_csv('Data/target_playlists.csv','\t')
tgt_tracks = pd.read_csv('Data/target_tracks.csv','\t')

In [3]:
train, test, tgt_tracks, tgt_playlists = rs.train_test_split_interface(data, 10, 20, 5, 2517)

100%|██████████| 45649/45649 [05:29<00:00, 138.38it/s]


# Models fitting

### Parameters

In [8]:
fit_dict_tsr = {'attributes' : ['artist_id', 'album', 'playcount'],
                'n_min_attr' : 90,
                'idf' : True,
                'measure' : 'dot',
                'shrinkage' : 0,
                'n_el_sim' : 65}

fit_dict_ibr = {'idf' : True,
                'measure' : 'dot',
                'shrinkage' : 10,
                'n_el_sim' : 65}

fit_dict_ubr = {'idf' : True,
            'measure' : 'imp_cos',
            'shrinkage' : 20,
            'n_el_sim' : 50}

alpha1=0.3
alpha2=0.2

beta1=0.6
beta2=0.01

In [10]:
rs = importlib.reload(rs)

In [11]:
ibr = IBR.ItemBasedRecommender(**fit_dict_ibr)
ibr.fit(tracks,train,tgt_tracks)

  0%|          | 34/45649 [00:00<02:15, 336.80it/s]

Calculated Indices


100%|██████████| 45649/45649 [05:10<00:00, 147.00it/s]


(45649, 100000)
Model URM built
Model URM regularized with IDF!


100%|██████████| 100000/100000 [1:04:23<00:00, 25.88it/s]


TypeError: 'coo_matrix' object is not subscriptable

In [ ]:
tsr= TSR.TopSimilarRecommender(**fit_dict_tsr)
tsr.fit(tracks,tgt_tracks)

In [ ]:
ubr = UBR.UserBasedRecommender(**fit_dict_ubr)
ubr.fit(tracks, train, tgt_playlists)

# Auxiliary structures

In [ ]:
_, IX_tgt_items, IX_tgt_playlists, _ = rs.create_sparse_indexes(playlists=tgt_playlists, tracks_reduced=tgt_tracks)

URM = rs.create_tgt_URM(IX_tgt_playlists, tsr.IX_items, train)
URM = URM.tocsr()

UBR_URM = rs.create_UBR_URM(ubr.IX_playlists, IX_tgt_items, train)
UBR_URM = UBR_URM.tocsr()

In [ ]:
ubr.S = ubr.S.T

In [ ]:
S_ensemble = rs.merge_similarities(tsr.S, ibr.S, beta1)

# Recommendation with only ratings level combination

In [ ]:
recommendetions = np.array([])
URM = URM.tocsr()
tsr.S = tsr.S.tocsr()
ibr.S = ibr.S.tocsr()
ubr.S = ubr.S.tocsr()
div_t = tsr.S.sum(axis=0)
div_i = ibr.S.sum(axis=0)
div_u = ubr.S.sum(axis=0)
H_t=10
H_i=30
H_u=20

for p in IX_tgt_playlists.values:
    avg_sims_t = (URM[p,:].dot(tsr.S).toarray().ravel())/(div_t+H_t)
    avg_sims_i = (URM[p,:].dot(ibr.S).toarray().ravel())/(div_i+H_i)
    avg_sims_u = (ubr.S[p,:].multiply(1/(div_u+H_u)).dot(UBR_URM).toarray().ravel())
    
    avg_sims = np.array(avg_sims_t*alpha1 + avg_sims_i*alpha2 + avg_sims_u*(1-alpha1-alpha2)).ravel()
    
    top = rs.top5_outside_playlist(avg_sims, p, train, IX_tgt_playlists, tsr.IX_tgt_items, False, False)
    recommendetions = np.append(recommendetions, rs.sub_format(top))
    if (p % 1000 == 0):
        print('Recommended ' + str(p) + ' users over ' + str(IX_tgt_playlists.values.shape[0]))

rec_only_ratings =  pd.DataFrame({'playlist_id' : IX_tgt_playlists.index.values, 'track_ids' : recommendetions})

# Recommendetion with S matrix and ratings level combination

In [ ]:
recommendetions = np.array([])
ubr.S = ubr.S.tocsr()
div_e = S_ensemble.sum(axis=0)
div_u = ubr.S.sum(axis=0)
H_e=30
H_u=20

for p in IX_tgt_playlists.values:
    avg_sims_e = (URM[p,:].dot(S_ensemble).toarray().ravel())#/(div_e+H_e)
    avg_sims_u = (ubr.S[p,:].multiply(1/(div_u+H_u)).dot(UBR_URM).toarray().ravel())
    
    avg_sims = np.array(avg_sims_e*beta2 + avg_sims_u*(1-beta2)).ravel()
    
    top = rs.top5_outside_playlist(avg_sims, p, train, IX_tgt_playlists, tsr.IX_tgt_items, False, False)
    recommendetions = np.append(recommendetions, rs.sub_format(top))
    if (p % 1000 == 0):
        print('Recommended ' + str(p) + ' users over ' + str(IX_tgt_playlists.values.shape[0]))

rec_bilevel =  pd.DataFrame({'playlist_id' : IX_tgt_playlists.index.values, 'track_ids' : recommendetions})

# Evaluation

In [ ]:
map_eval_only_ratings = rs.evaluate(rec_only_ratings, test, 'MAP')

In [ ]:
map_eval_bilevel = rs.evaluate(rec_bilevel, test, 'MAP')

In [ ]:
print(map_eval_only_ratings)
print(map_eval_bilevel)

# Saving run data

In [ ]:
run_data = {'recommender_1' : tsr.__class__.__name__,
            'recommender_2' : ibr.__class__.__name__,
            'recommender_3' : ubr.__class__.__name__,
            'fit_parameters_1' : fit_dict_tsr,
            'fit_parameters_2' : fit_dict_ibr,
            'fit_parameters_3' : fit_dict_ubr,
            'alpha1_only_ratings' : alpha1,
            'alpha2_only_ratings' : alpha2,
            'beta1_bilevel' : beta1,
            'beta2_bilevel' : beta2,
            'evaluation_only_ratings' : map_eval_only_ratings,
            'evaluation_bilevel' : map_eval_bilevel}

In [ ]:
with open('runs_data.json', 'a') as fp:
    json.dump(run_data, fp, indent=2)
    fp.write('\n')